<a href="https://colab.research.google.com/github/mshchurska/mariia_shchurska/blob/main/Copy_of_10_2_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 10.2

> Replace all TODOs with your code. Do not change any other code.

In [ ]:
# Do not edit this cell

import csv
import unittest


## Clean code

### Task 1

You are given a function that reads a csv file with temperature measurements (see example below), converts Fahrenheit values to Celsius, calculates and prints some statistics, and writes to another file. It looks a bit messy, let's clean it up!

Example file:
```csv
Temperature (F)
78.5
81.2
75.9
82.1
```

Do the steps below one by one, editing the code in the cell:
1. Naming is so ambiguous and unclear, let's rename variables and function name with proper names.
2. Are these comments really useful?
3. This function does quite a lot, let's divide it in the way that each function does only one thing, and there's one main function that uses others.
4. There seem to be some magic coefficients in the temperature conversion part; let's make them obvious.

If you find any additional improvements, feel free to implement them and leave a comment under your code with an explanation.

In [ ]:
import csv

def fahrenheit_to_celsius(f):
    return (f - 32) * 5 / 9

def read_temperatures(file_name):
    celsius_list = []
    with open(file_name, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # пропускаємо заголовок
        for row in reader:
            f = float(row[0])
            c = fahrenheit_to_celsius(f)
            celsius_list.append(c)
    return celsius_list

def calculate_stats(temperatures):
    avg = sum(temperatures) / len(temperatures)
    return avg, min(temperatures), max(temperatures)

def write_temperatures(file_name, temperatures):
    with open(file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Temperature (C)'])
        for t in temperatures:
            writer.writerow([t])

def main(input_file, output_file):
    temps = read_temperatures(input_file)
    avg, min_temp, max_temp = calculate_stats(temps)

    print("Statistics:")
    print(f"Average: {avg:.2f}°C")
    print(f"Minimum: {min_temp:.2f}°C")
    print(f"Maximum: {max_temp:.2f}°C")

    write_temperatures(output_file, temps)


### Task 2

How would you write tests for the initial implementation? What exactly would you test in the function?

I hope you see now that once functionality is separated, it's easier to test it in isolation. So, let's write a couple of unit tests for your function and one integration test for your main function.

Hint: you would probably want to mock reading from/writing to file to make the test independent from the environment.

In [ ]:
import unittest
from unittest.mock import mock_open, patch
from io import StringIO

class UnitTestCase(unittest.TestCase):

    def test_fahrenheit_to_celsius(self):
        from my_module import fahrenheit_to_celsius
        self.assertAlmostEqual(fahrenheit_to_celsius(32), 0)
        self.assertAlmostEqual(fahrenheit_to_celsius(212), 100)

    def test_calculate_stats(self):
        from my_module import calculate_stats
        data = [10.0, 20.0, 30.0]
        avg, min_t, max_t = calculate_stats(data)
        self.assertEqual(avg, 20.0)
        self.assertEqual(min_t, 10.0)
        self.assertEqual(max_t, 30.0)

    @patch("builtins.open", new_callable=mock_open, read_data="Temperature (F)\n32\n212\n")
    def test_read_temperatures(self, mock_file):
        from my_module import read_temperatures
        result = read_temperatures("fake.csv")
        self.assertAlmostEqual(result[0], 0.0)
        self.assertAlmostEqual(result[1], 100.0)



class IntegrationTestCase(unittest.TestCase):

    @patch("builtins.open", new_callable=mock_open)
    @patch("csv.writer")
    @patch("csv.reader", return_value=[["Temperature (F)"], ["32"], ["212"]])
    def test_main_function(self, mock_reader, mock_writer, mock_file):
        from my_module import main
        main("input.csv", "output.csv")

        # Перевіряємо, що запис відбувся у файл
        mock_file.assert_called()
        mock_writer().writerow.assert_any_call(['Temperature (C)'])
        mock_writer().writerow.assert_any_call([0.0])
        mock_writer().writerow.assert_any_call([100.0])

